# Clustering Crypto

In [125]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [126]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv")
crypto_df = crypto_df.set_index("Unnamed: 0"," ")
#crypto_df = crypto_df.columns.Unnamed: 0 = None
#crypto_df = crypto_df.rename_axis(None, axis = 1)
#del crypto_df.index.name
crypto_df.index.names = ['']
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [127]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [128]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().sum()

0

In [129]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=["IsTrading"])

crypto_df.head(10)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [130]:
# Remove rows that have at least 1 null value.
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")
    
    
    


Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [131]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [132]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cryptocurrencies_name = crypto_df[["CoinName"]]
cryptocurrencies_name.head()

,CoinName
,
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [133]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df[["Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply"]].copy()
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0


In [134]:
# Use get_dummies() to create variables for text features.
#X = pd.get_dummies(crypto_df)
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head(10)
#y = crypto_df['loan_status']

(685, 109)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.10282804 -0.03823841 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.05411338 -0.07664017 -0.03823596 -0.05411338 -0.05411338 -0.03823596
  -0.03823596 -0.18216065 -0.05411338 -0.03823596 -0.03823596 -0.08574929
  -0.03823596 -0.10160947 -0.06632365 -0.03823596 -0.03823596 -0.1642757
  -0.03823596 -0.03823596 -0.13908716 -0.03823596 -0.03823596 -0.07664017
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.06632365 -0.03823596
  -0.07664017 -0.08574929 -0.07664017 -0.03823596 -0.03823596 -0.12775161
  -0.1335313  -0.13908716 -0.03823596 -0.05411338 -0.03823596 -0.06632365
  -0.1689039  -0.03823596 -0.03823596 -0.03823596 -0.07664017 -0.17342199
  -0.33468341 -0.03823596 -0.08574929 -0.06632365 -0.05411338 -0.03823596
   1.42042992 -0.06632365 -0.03823596 -0.03823596 -0.06632365 -0.06632365
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.03823596 -0.41586681 -0.03823596 -0.19054822 -0.03823596 -0.10870529
  -0.07664017 -0.09400279 -0.03823596 -

### Deliverable 2: Reducing Data Dimensions Using PCA

In [136]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [137]:
# Create a DataFrame with the three principal components.
pca_df =pd.DataFrame (data=crypto_pca,columns=["PC1","PC2","PC3"],index=crypto_df.index)
pca_df.head(10)

,PC1,PC2,PC3
,,,
42,-0.237011,1.193314,-0.451605
404,-0.222535,1.191985,-0.451983
808,-0.159145,0.833268,-0.221968
1337,0.280074,1.954540,-0.403649
BTC,-0.203428,-1.416650,0.145584
ETH,-0.271954,-2.098155,0.247414
LTC,-0.280675,-1.056661,-0.084069
DASH,-0.226878,1.426893,-0.339785
XMR,-0.260225,-2.151619,0.293356


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [138]:
# Create an elbow curve to find the best value for K.
# Initialize empty inertia list
inertia = []
# Initialize k range
k = list(range(1,11))
# Looping through k list
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


C:\Users\vanda\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [139]:
# Initialize the K-Means model.
km4 = KMeans(n_clusters=4, random_state=0)
# Fit the mode
km4.fit(pca_df)
# Predict clusters
predictions = km4.predict(pca_df)
predictions

array([0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 0,
       0, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 3, 0,
       0, 3, 0, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3,
       3, 3, 3, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0,
       3, 0, 0, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0,
       0, 0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 3, 0, 3, 0, 0, 3, 0, 0, 0,
       3, 3, 0, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0,
       3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 3, 3, 0, 0, 3, 0, 0, 0,
       3, 3, 3, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0,

In [140]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df], axis=1, join='inner')


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = cryptocurrencies_name.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = km4.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(685, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.237011,1.193314,-0.451605,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.222535,1.191985,-0.451983,404Coin,0
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.159145,0.833268,-0.221968,808,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.280074,1.954540,-0.403649,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.203428,-1.416650,0.145584,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.271954,-2.098155,0.247414,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.280675,-1.056661,-0.084069,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.226878,1.426893,-0.339785,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.260225,-2.151619,0.293356,Monero,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [141]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC1", y="PC2", z="PC3", color="Class", symbol="Class", width=800, 
                    hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [142]:
# Create a table with tradable cryptocurrencies.
tradable_crypto_table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 
                                                  'TotalCoinsMined', 'Class'], sortable=True, selectable=True)
tradable_crypto_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [143]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

There are 685 tradable cryptocurrencies.


In [144]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

array([[4.55364914e-16, 5.94230127e-03],
       [5.76795558e-09, 7.00182308e-03],
       [0.00000000e+00, 5.94230122e-03],
       ...,
       [1.51812440e-08, 6.92655266e-03],
       [2.27682457e-10, 5.94962775e-03],
       [1.08420218e-11, 5.94243008e-03]])

In [145]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.

plot_df['CoinName'] = clustered_df.CoinName
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
,,,,
42,4.553649e-16,0.005942,42 Coin,0
404,5.767956e-09,0.007002,404Coin,0
808,0.000000e+00,0.005942,808,0
1337,3.406122e-06,0.035342,EliteCoin,0
BTC,2.276825e-10,0.005960,Bitcoin,3
ETH,0.000000e+00,0.006050,Ethereum,3
LTC,9.107298e-10,0.006006,Litecoin,3
DASH,2.385245e-10,0.005951,Dash,0
XMR,0.000000e+00,0.005960,Monero,3


In [148]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', hover_cols='CoinName', by='Class')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)